In [1]:
import xml.etree.ElementTree as Xet
import pandas as pd
import numpy as np
import geopandas as gpd
import os
import matplotlib.pyplot as plt
import warnings
import re

# Ignore all warnings
warnings.filterwarnings('ignore')

# Arizona 2022 General Election PBER

## Load election results

Data from openelections, can be downloaded from Github at https://github.com/openelections/openelections-data-az/tree/master/2022

In [2]:
csv_path = './20221108__az__general__precinct.csv'
pd.set_option('display.max_rows', 0)
pd.set_option('display.max_columns', 0)
election_results = pd.read_csv(csv_path)
election_results.head(5)

FileNotFoundError: [Errno 2] No such file or directory: './20221108__az__general__precinct.csv'

Another dataset from MEDSL, just for double check, in cases where openelection data does not seem correct

In [ ]:
csv_path_az = '../AZ-cleaned.csv'
election_results_MEDSL = pd.read_csv(csv_path_az)
election_results_MEDSL.head(5)

### Process data

In [ ]:
rows_before = election_results.shape[0]

# Formate votes to numbers
election_results['votes'] = pd.to_numeric(election_results['votes'], errors='coerce')

# Remove rows without a candidate
election_results_cleaned = election_results.dropna(subset=['candidate'])

rows_after = election_results_cleaned.shape[0]

# Remove rows without a valid vote
election_results_cleaned = election_results_cleaned[election_results_cleaned['votes'] > 0]

print(f"Number of rows before cleaning: {rows_before}")
print(f"Number of rows after cleaning: {rows_after}")

# Convert data types 
election_results_cleaned['votes'] = election_results_cleaned['votes'].astype(int)

# Display the cleaned data
election_results_cleaned.tail(5)

In [ ]:
# Function to swap the parts of the name
def swap_name(name):
    # add a try catach
    try:
        parts = name.split(',')
        if len(parts) == 2:
            return parts[1].strip() + ' ' + parts[0].strip()
    except:
        return name
    return name

# Convert the 'party' column to strings, handling NaN values by filling them with an empty string or a placeholder
election_results_cleaned['party'] = election_results_cleaned['party'].astype(str)

# Clean and format
election_results_cleaned['party'] = election_results_cleaned['party'].str.strip().str.upper()
election_results_cleaned['office'] = election_results_cleaned['office'].str.strip().str.upper()
election_results_cleaned['candidate'] = election_results_cleaned['candidate'].str.strip().str.upper()
election_results_cleaned['candidate'] = election_results_cleaned['candidate'].apply(swap_name)

# Display the unique values for 'office', 'candidate', and 'party'
unique_offices = sorted(election_results_cleaned['office'].unique())
unique_candidates = sorted(election_results_cleaned['candidate'].unique())
unique_parties = sorted(election_results_cleaned['party'].unique())

print(f"Unique values in 'office':\n{unique_offices}")
print(f"\nUnique values in 'candidate':\n{unique_candidates}")
print(f"\nUnique values in 'party':\n{unique_parties}")

# Count the number of unique candidates for each office
unique_candidates_per_office = election_results_cleaned.groupby('office')['candidate'].nunique()

print("Number of unique candidates for each office:")
print(unique_candidates_per_office)

Just by looking at the values:


For values in 'office', they seem pretty correct  
  
For candidates, there exist: 'WRITE-IN', 'WRITE-INS', 'UNDER VOTES', 'OVER VOTES', 'NOT QUALIFIED'  
And these names look suspicious:  
'T., TOM'  
'QUEZADA, MARTÍN'  
'MERRILL, RAYSHAWN D'ANTHONY "SHAWN"'  
'MARTÍN QUEZADA' should be 'MARTÍN QUEZADA'  
'MARKY KELLY' should be 'MARK KELLY'  
'LUTES-BURTON, MIKAELA SHONNIE "MIKKI"'  
'JACQUELINE PARKER', 'JACUELINE PARKER'  
'HODGE, JEVIN D.'  
'HANS, CYNTHIA "CINDY"'  
'HAMADEH, ABRAHAM "ABE"'  
'GRIJALVA, RAUL', 'GRIJALVA, RAUL 32', 'GRIJALVA, RAÚL', 'GRIJALVA, RAÖL', 'GRIJALVA, RAÖL 160'  
'GARCIA SNYDER, GARY', 'GARY GARCIA SNYDER'  
'GABALDON, ROSANNA', 'GABALDÓN, ROSANNA'  
'EPSTEIN, DENISE "MITZI"'  
'DUNN, TIMOTHY "TIM"'  
'DI GENOVA, TRISTA', 'DI GENOVA, TRISTA "TRISTA"'  
'DENNY, STEPH NOELLE "STEPH"'  
'CHASTON, JAMES "JIM"'  
'BORDEN, DEBRA JO "D-JO"'  
'ABRAHAM "ABE" HAMADEH', 'ABRAHAM HAMADEH'  
  
For values in party, 'WRITE IN DEM' should be 'DEM', 'LBT' should be 'LIB'


In [ ]:
# Get rid of rows where candidate is in 'UNDER VOTES', 'OVER VOTES', 'NOT QUALIFIED'
exclude_candidates = ['UNDER VOTES', 'OVER VOTES', 'NOT QUALIFIED']
election_results_cleaned = election_results_cleaned[~election_results_cleaned['candidate'].isin(exclude_candidates)]

# replace candidate with 'WRITE-INS' with 'WRITE-IN'
election_results_cleaned['candidate'] = election_results_cleaned['candidate'].replace('WRITE-INS', 'WRITE-IN')

# Categorize duplicated parties
election_results_cleaned['party'] = election_results_cleaned['party'].replace({
    'WRITE IN DEM': 'DEM',
    'LBT': 'LIB',
})

# Replace NA values in 'party' column with 'Other'
election_results_cleaned['party'] = election_results_cleaned['party'].fillna('N')


Use algorithms to find potential duplicates

In [ ]:
from collections import defaultdict
from fuzzywuzzy import fuzz, process

candidates = ['ABRAHAM "ABE" HAMADEH', 'ABRAHAM HAMADEH', 'ADRIAN FONTES', 'AGUILAR, CESAR', 'ALAN SMITH', 'ALICE NOVOA', 'ALMA HERNANDEZ', 'ALSTON, LELA', 'ANDRES CANO', 'ANDY BIGGS', 'ANTHONY CAMBONI', 'AUSTIN, LORENA', 'BARBARA ROWLEY PARKER', 'BARRAZA, BRITTANI', 'BARTO, NANCY', 'BIASIUCCI, LEO', 'BIGGS, ANDY', 'BLAKE MASTERS', 'BLATTMAN, SETH', 'BORDEN, DEBRA JO "D-JO"', 'BORDES, SHERRISE', 'BORRELLI, SONNY', 'BRANNIES, MARYN M.', 'BRAVO, FLAVIO', 'BRIAN FERNANDEZ', 'BRIAN RADFORD', 'BULLOCK, CHRISTOPHER', 'BURCH, EVA', 'CAMBONI, ANTHONY', 'CARBONE, MICHAEL', 'CARROLL, FRANK', 'CARTER, NEAL', 'CASTEEN, JEANNE', 'CATHY RANSOM', 'CHAPLIK, JOSEPH', 'CHASTON, JAMES "JIM"', 'CHAVIRA CONTRERAS, LUPE', 'CHRIS MATHIS', 'CHRIS SARAPPO', 'CHRISTOPHER BULLOCK', 'CLARK, SANDA', 'CLEVELAND, JIM', 'CLINT WILLIAM SMITH', 'CONSUELO HERNANDEZ', 'CONTRERAS, PATRICIA "PATTY"', 'COOPER, KELLY', 'CORY MCGARR', 'CRANE, ELI', 'DAMIEN KENNEDY', 'DANA ALLMOND', 'DARROW, CADEN', 'DAVID CHRISTIAN FARNSWORTH', 'DAVID COOK', 'DAVID GOWAN', 'DAVID MARSHALL SR.', 'DAVIDA, EDWARD', 'DE LOS SANTOS, OSCAR', 'DEBORAH MCEWEN', 'DENNY, STEPH NOELLE "STEPH"', 'DI GENOVA, TRISTA', 'DI GENOVA, TRISTA "TRISTA"', 'DIAZ, LUPE', 'DUGGER, THOMAS', 'DUNN, TIMOTHY "TIM"', 'ELI CRANE', 'EPSTEIN, DENISE "MITZI"', 'FARNSWORTH, DAVID CHRISTIAN', 'FERNANDEZ, BRIAN', 'FIERRO, NICK', 'FINCHEM, MARK', 'FONTES, ADRIAN', 'GABALDON, ROSANNA', 'GABALDÓN, ROSANNA', 'GABALD√ÌN, ROSANNA', 'GAIL GRIFFIN', 'GALLEGO, RUBEN', 'GARCIA SNYDER, GARY', 'GARY GARCIA SNYDER', 'GILLETTE, JOHN', 'GOSAR, PAUL', 'GOWAN, DAVID', 'GRANTHAM, TRAVIS', 'GRAYSON, RICHARD', 'GRESS, MATT', 'GRIFFIN, GAIL', 'GRIJALVA, RAUL', 'GRIJALVA, RAUL 32', 'GRIJALVA, RAÚL', 'GRIJALVA, RA√ÖL', 'GRIJALVA, RA√ÖL 160', 'GUZMAN, ALIXANDRIA', 'HAMADEH, ABRAHAM "ABE"', 'HANS, CYNTHIA "CINDY"', 'HARRIS, LIZ', 'HEAP, JUSTIN', 'HELEN HUNTER', 'HENDRIX, LAURIN', 'HERNANDEZ, ANNA', 'HERNANDEZ, CONSUELO', 'HERNANDEZ, LYDIA', 'HERNANDEZ, MELODY', 'HOBBS, KATIE', 'HODGE, JEVIN D.', 'HOFFMAN, JAKE', 'HOLBROOK, STEPHANIE BLAIR', 'HOLZAPFEL, ROXANA', 'HUDELSON, ROB', 'HUNTER, HELEN', 'JACQUELINE PARKER', 'JACUELINE PARKER', 'JAKE HOFFMAN', 'JAVIER GARCIA RAMOS', 'JESUS LUGO JR.', 'JIM CLEVELAND', 'JOHN GILLETTE', 'JONES, STEPHAN "STEVE"', 'JUAN CISCOMANI', 'JUAN CISOMANI', 'JUSTIN HEAP', 'JUSTINE WADSACK', 'KAISER, STEVE', 'KARI LAKE', 'KATIE HOBBS', 'KAVANAGH, JOHN', 'KEITH SEAMAN', 'KELLY, MARK', 'KEN BENNETT', 'KENNEDY, DAMIEN', 'KENNEDY, SANDRA', 'KERBY, TAYLOR', 'KERN, ANTHONY', 'KERR, SINE', 'KEVIN THOMPSON', 'KIMBERLY YEE', 'KIRSTEN ENGEL', 'KISSINGER, DON', 'KOLODIN, ALEXANDER', 'KRIS MAYES', 'KRISTEN ENGEL', 'KUBY, LAUREN', 'KYLE NITSCHKE', 'LAKE, KARI', 'LAMAR, CHRISTIAN', 'LAUREN KUBY', 'LEO BIASIUCCI', 'LEO BISIUCCI', 'LESKO, DEBBIE', 'LESTER MAUL', 'LINDA EVANS', 'LIVINGSTON, DAVID', 'LONGDON, JENNIFER', 'LOUGHRIGE, BILL', 'LUGO, JESUS JR.', 'LUIS POZZOLO', 'LUPE DIAZ', 'LUTES-BURTON, MIKAELA SHONNIE  "MIKKI"', 'MAE PESHLAKAI', 'MARC J. VICTOR', 'MARIANA SANDOVAL', 'MARK FINCHEM', 'MARK KELLY', 'MARKY KELLY', 'MARSH, CHRISTINE', 'MARSH, PAUL', 'MARTIN QUEZADA', 'MARTINEZ, TERESA', 'MARTÍN QUEZADA', 'MART„N QUEZADA', 'MASTERS, BLAKE', 'MAUL, LESTER "SKIP"1', 'MAYES, KRIS', 'MCEWEN, DEBORAH', 'MCLEAN, TY RICHARD JR.', 'MENDEZ, JUAN', 'MENDOZA, MARY ANN', 'MERRILL, RAYSHAWN D\'ANTHONY "SHAWN"', 'MESNARD, J.D.', 'MICHAEL CARBONE', 'MICHELE PENA', 'MIKAELA LUTES-BURTON', 'MIKE FOGEL', 'MIKE NICKERSON', 'MIRANDA, CATHERINE', 'MONTENEGRO, STEVE', 'MYERS, NICHOLAS "NICK"', 'MYRON TSOSIE', 'NANCY GUTIERREZ', 'NEAL CARTER', 'NEIL SINCLAIR', 'NICHOLAS "NICK" MYERS', 'NICK FIERRO', 'NICK MYERS', 'NOT QUALIFIED', 'NOVOA, ALICE', "O'HALLERAN, TOM", 'ORTIZ, ANALISE', 'OVER VOTES', 'PARKER, BARBARA ROWLEY', 'PARKER, JACQUELINE', 'PAUL GOSAR', 'PAUL MARSH', 'PAWLIK, JENNIFER', 'PAYNE, KEVIN', 'PEARCE, KATHY', 'PENA, MICHELE', 'PETERSEN, WARREN', 'PEÑA M., TATIANA', 'PINGERELLI, BEVERLY', 'PODEYN, SCOTT', 'POUNDS, WILLIAM JOSUÉ IV', 'POZZOLO, LUIS', 'PRIYA SUNDARESHAN', 'QUANG NGUYEN', 'QUEZADA, MARTIN', 'QUEZADA, MARTÍN', 'QUEZADA, MART√ÇN', 'QUINONEZ, MARCELINO', 'RACHEL JONES', 'RAMOS, JAVIER GARCIA', 'RAUL GRIJALVA', 'RAYMER, DAVID', 'RAYSHAWN MERRILL', 'REESE, BRANDY', 'RENEE ROXANNE', 'RICHARD TY MCLEAN, JR.', 'RICHARDSON, DAVID WAYNE', 'ROB HUDELSON', 'RODRIGUEZ, ROXANNE RENEE', 'ROE, TERRY', 'ROSANNA GABALDON', 'SALLY ANN GONZALES', 'SALMAN, ATHENA', 'SAMANTHA SEVERSON', 'SANDA CLARK', 'SANDOVAL, DAVID', 'SANDOVAL, MARIANA', 'SANDRA KENNEDY', 'SCANTLEBURY, ROBERT', 'SCHWEIKERT, DAVID', 'SCHWIEBERT, JUDY', 'SEAMAN, KEITH', 'SELINA BLISS', 'SEVERSON, SAMANTHA', 'SHAH, AMISH', 'SHAMP, JANAE', 'SHERRISE BORDES', 'SHOPE, THOMAS "T.J."', 'SILVEY, JEFF', 'SINE KERR', 'SMELTZER, TODD JAMES', 'SMITH, ALAN', 'SMITH, AUSTIN', 'SMITH, CLINT WILLIAM', 'SONNY BORRELLI', 'SPREITZER, JEREMY', 'STAHL HAMILTON, STEPHANIE', 'STAN CAINE', 'STANTON, GREG', 'STEPH NOELLE DENNY', 'STEPHANIE STAHL HAMILTON', 'SUN, LEEZAH ELSA', 'SYMS, MARIA', 'T., TOM', 'TAYLOR KERBY', 'TAYLOR, WILLIAM MICHAEL "WILL"', 'TERECH, LAURA', 'TERESA MARTINEZ', 'TERÁN, RAQUEL', 'THERESA HATATHLIE', 'THOMAS "T.J." SHOPE', 'THOMAS SHOPE', 'THOMPSON, KEVIN', 'TIMOTHY "TIM" DUNN', "TOM O'HALLERAN", 'TOMA, BEN', 'TRAVERS, ANASTASIA "STACEY"', 'TREADWELL, JENNIFER "JENN"', 'UNDER VOTES', 'VICTOR, MARC J.', 'WENDY ROGERS', 'WEST, LIANA', 'WILLIAM JOSUE POUNDS IV', 'WILLOUGHBY, JULIE', 'WILMETH, JUSTIN', 'WRITE-IN', 'WRITE-INS', 'YEE, KIMBERLY', 'ZINK, JEFF NELSON']

# Function to normalize names by removing punctuation and lowercasing
def normalize_name(name):
    name = re.sub(r'[^\w\s]', '', name)  # Remove punctuation
    return name.lower().strip()  # Convert to lowercase and strip whitespace
# Normalize all names
normalized_candidates = [normalize_name(name) for name in candidates]

# Dictionary to store potential duplicates
potential_duplicates = defaultdict(list)

# Use fuzzy matching to identify potential duplicates within the list
for i, name in enumerate(normalized_candidates):
    matches = process.extract(name, normalized_candidates, limit=len(normalized_candidates))
    for match in matches:
        # Ensure match isn't the same name and check similarity score
        if match[1] > 85 and match[0] != name:
            potential_duplicates[name].append((candidates[i], match))

# Print potential duplicates
print("Potential duplicates:")
duplicates_found = False
for normalized_name, matches in potential_duplicates.items():
    if len(matches) > 1:  # More than one match indicates potential duplicates
        duplicates_found = True
        print(f"Normalized Name: {normalized_name}")
        for original, (match_name, score) in matches:
            print(f"  - Original: {original} is similar to: {candidates[normalized_candidates.index(match_name)]} with score {score}")

if not duplicates_found:
    print("No duplicates found.")

In [ ]:
# Dictionary mapping old candidate names to new candidate names
replacement_dict = {
    'ABRAHAM "ABE" HAMADEH': 'ABRAHAM HAMADEH',
    'HAMADEH, ABRAHAM "ABE"': 'ABRAHAM HAMADEH',
    'BORDEN, DEBRA JO "D-JO"': 'DEBRA JO BORDEN',
    'CHASTON, JAMES "JIM"': 'CHASTON, JAMES',
    'DENNY, STEPH NOELLE "STEPH"': 'DENNY, STEPH NOELLE',
    'DUNN, TIMOTHY "TIM"': 'DUNN TIMOTHY',
    'EPSTEIN, DENISE "MITZI"': 'EPSTEIN, DENISE',
    'GABALDON, ROSANNA': 'GABALDÓN, ROSANNA',
    'GARCIA SNYDER, GARY': 'GARY, GARCIA SNYDER',
    'GRIJALVA, RAUL 32': 'RAUL GRIJALVA',
    'GRIJALVA, RAÚL': 'RAUL GRIJALVA',
    'GRIJALVA, RAÖL': 'RAUL GRIJALVA',
    'GRIJALVA, RAÖL 160': 'RAUL GRIJALVA',
    'HANS, CYNTHIA "CINDY"': 'HANS, CYNTHIA',
    'JACUELINE PARKER': 'JACQUELINE PARKER',
    'LUTES-BURTON, MIKAELA SHONNIE "MIKKI"': 'MIKARLA LUTES-BURTON', 
    'MARKY KELLY': 'MARK KELLY',
    'MART„N QUEZADA': 'MARTÍN QUEZADA',
    'QUEZADA, MART√ÇN':'MARTÍN QUEZADA',
    'MERRILL, RAYSHAWN D\'ANTHONY "SHAWN"': 'RAYSHAWN, MERRILL',
    'MAUL, LESTER "SKIP"1': 'MAUL, LESTER',
    'BRANNIES, MARYN M.': 'MARYN, BRANNIES',
    'CHAVIRA CONTRERAS, LUPE': 'LUPE, CONTRERAS',
    'CONTRERAS, PATRICIA "PATTY"': 'PATRICIA CONTRERAS',
    'DI GENOVA, TRISTA "TRISTA"': 'DI GENOVA, TRISTA',
    'GABALD√ÌN, ROSANNA': 'GABALDÓN, ROSANNA',
    'GARY, GARCIA SNYDER': 'GARY GARCIA SNYDER',
    'GRIJALVA, RA√ÖL': 'RAUL GRIJALVA',
    'GRIJALVA, RA√ÖL 160': 'RAUL GRIJALVA',
    'GRIJALVA, RAUL': 'RAUL GRIJALVA',
    'HODGE, JEVIN D.': 'JEVIN, HODGE',
    'JONES, STEPHAN "STEVE"': 'JONES STEPHAN',
    'JUAN CISOMANI': 'JUAN CISCOMANI',
    'LEO BISIUCCI': 'LEO BIASIUCCI',
    'MARTIN QUEZADA': 'MARTÍN QUEZADA',
    'QUEZADA, MARTIN': 'MARTÍN QUEZADA',
    'QUEZADA, MARTÍN': 'MARTÍN QUEZADA',
    'FONTES, ADRIAN': 'ADRIAN FONTES',
    'AGUILAR, CESAR': 'CESAR AGUILAR',
    'SMITH, ALAN': 'ALAN SMITH',
    'NOVOA, ALICE': 'ALICE NOVOA',
    'ALSTON, LELA': 'LELA ALSTON',
    'BIGGS, ANDY': 'ANDY BIGGS',
    'CAMBONI, ANTHONY': 'ANTHONY CAMBONI',
    'AUSTIN, LORENA': 'LORENA AUSTIN',
    'BARBARA ROWLEY PARKER': 'BARBARA PARKER',
    'PARKER, BARBARA ROWLEY': 'BARBARA PARKER',
    'BARRAZA, BRITTANI': 'BRITTANI, BARRAZA',
    'BARTO, NANCY': 'NANCY BARTO',
    'BIASIUCCI, LEO': 'LEO BIASIUCCI',
    'MASTERS, BLAKE': 'BLAKE MASTERS',
    'BLATTMAN, SETH': 'SETH BLATTMAN',
    'BORDES, SHERRISE': 'SHERRISE BORDES',
    'BORRELLI, SONNY': 'SONNY BORRELLI',
    'BRAVO, FLAVIO': 'FLAVIO BRAVO',
    'FERNANDEZ, BRIAN': 'BRIAN FERNANDEZ',
    'BULLOCK, CHRISTOPHER': 'CHRISTOPHER BULLOCK',
    'BURCH, EVA': 'EVA BURCH',
    'CARBONE, MICHAEL': 'MICHAEL CARBONE',
    'CARROLL, FRANK': 'FRANK CARROLL',
    'CARTER, NEAL': 'NEAL CARTER',
    'CASTEEN, JEANNE': 'JEANNE CASTEEN',
    'JESUS JR. LUGO': 'JESUS LUGO',
    'MIKAELA SHONNIE  "MIKKI" LUTES-BURTON': 'MIKAELA LUTES-BURTON',
    'NICHOLAS "NICK" MYERS': 'NICK MYERS',
    'ROSANNA GABALDON': 'ROSANNA GABALDÓN', 
    'TATIANA PEÑA M.': 'TATIANA PEÑA',
    'THOMAS "T.J." SHOPE': 'THOMAS SHOPE',
    'TY RICHARD JR. MCLEAN': 'TY MCLEAN',
    'WILLIAM JOSUE POUNDS IV': 'WILLIAM POUNDS',
    'WILLIAM JOSUÉ IV POUNDS': 'WILLIAM POUNDS',
    'RA√ÖL GRIJALVA': 'RAUL GRIJALVA',
    'ROSANNA GABALD√ÌN': 'ROSANNA GABALDÓN',
    'RAÚL GRIJALVA': 'RAUL GRIJALVA',
    'RA√ÖL 160 GRIJALVA': 'RAUL GRIJALVA',
    'RA√ÖL GRIJALVA': 'RAUL GRIJALVA',
    'RAUL 32 GRIJALVA': 'RAUL GRIJALVA',
    'STEPH NOELLE "STEPH" DENNY': 'STEPH NOELLE DENNY',
    'MART√ÇN QUEZADA': 'MARTÍN QUEZADA',
    'JR. RICHARD TY MCLEAN': 'TY MCLEAN',
    'RAYSHAWN D\'ANTHONY "SHAWN" MERRILL': 'RAYSHAWN MERRILL',
    'KRISTEN ENGEL': 'KIRSTEN ENGEL',
    'TOM T.': 'WRITE-IN',
    'WRITE_IN': 'WRITE-IN',
}

# Replace old candidate names with new names
election_results_cleaned['candidate'] = election_results_cleaned['candidate'].replace(replacement_dict)

unique_candidates_after_mapping = sorted(election_results_cleaned['candidate'].unique())
unique_candidates_after_mapping

everything now looks right, besides this guy 'TOM T.'

In [ ]:
# Filter rows where candidate is 'TOM T.'
tom_t_rows = election_results_cleaned[election_results_cleaned['candidate'] == 'TOM T.']

tom_t_rows


After searching, there is no such person as Tom T and he had no party affiliation, only Tom O'Halleran who runs for U.S. Representative in Congress - District No. 2. So change this Tom T to write-in.

In [ ]:
# Print the names of the columns in the final DataFrame
print("Column names in the final DataFrame:")
print(final_df.columns.tolist())

In [ ]:
# Group by office, district, and candidate, and sum the votes
office_district_candidate_votes = election_results_cleaned.groupby(['office', 'district', 'candidate'])['votes'].sum()

# Print the number of votes for each candidate
for (office, district, candidate), votes in office_district_candidate_votes.items():
    district_info = f"District {int(district)}" if pd.notna(district) else "At Large"
    print(f"Office: {office} - {district_info} - Candidate: {candidate}, Votes: {votes}")


## Process Election Data

In [ ]:
election_results_cleaned.columns = list

In [ ]:
# Create a mapping for 'office'
office_mapping = {
    'ATTORNEY GENERAL': 'ATG',
    'CORPORATION COMMISSIONER': 'COC',
    'GOVERNOR': 'GOV',
    'SECRETARY OF STATE': 'SOS',
    'STATE HOUSE': 'SL',
    'STATE MINE INSPECTOR': 'STM',
    'STATE SENATE': 'SU',
    'STATE TREASURER': 'STT',
    'U.S. HOUSE': 'USH',
    'U.S. SENATE': 'USS'
}


# Find the largest district number
max_district = int(election_results_cleaned['district'].max())

# Determine the number of digits for district formatting
num_digits = len(str(max_district))

# Function to create a unique column name and print its meaning
def create_column_name(row, existing_names, num_digits):
    office_abbr = office_mapping.get(row['office'], '')
    party_initial = row['party'][0]
    last_name = row['candidate'].split()[-1][:3].upper()
    
    if pd.notna(row['district']):
        district = str(int(row['district'])).zfill(num_digits)  # Ensure district has leading zeros
        base_name = f"G{office_abbr}{district}{party_initial}{last_name}"
    else:
        base_name = f"G22{office_abbr}{party_initial}{last_name}"
    
    # Ensure the column name is unique
    if base_name not in existing_names:
        unique_name = base_name
    else:
        suffix = 1
        unique_name = f"{base_name}_{suffix}"
        while unique_name in existing_names:
            suffix += 1
            unique_name = f"{base_name}_{suffix}"
    
    # Print the meaning of the generated column name
    district_info = f"District {district}" if pd.notna(row['district']) else "General Election Year 2022"
    print(f"{unique_name}: Office - {row['office']}, {district_info}, Party - {row['party']}, Candidate - {row['candidate']}")

    return unique_name


# Get unique combinations of counties and precincts
unique_combinations = election_results_cleaned[['county', 'precinct']].drop_duplicates()

# Initialize an empty list to store the results
results = []

# Iterate over each unique combination of county and precinct
for _, unique_row in unique_combinations.iterrows():
    county = unique_row['county']
    precinct = unique_row['precinct']
    
    # Filter for the specific county and precinct
    filtered_df = election_results_cleaned[(election_results_cleaned['county'] == county) & (election_results_cleaned['precinct'] == precinct)]
    
    # Track existing column names to ensure uniqueness
    existing_names = set()
    
    # Apply the function to create new column names
    filtered_df['ColumnName'] = filtered_df.apply(lambda row: create_column_name(row, existing_names, num_digits), axis=1)
    
    # Update the set of existing names
    existing_names.update(filtered_df['ColumnName'].tolist())
    
    # Pivot dataset so that each row represents a precinct and each column represents a candidate
    pivoted_df = filtered_df.pivot_table(
        index=['county', 'precinct'],
        columns='ColumnName',
        values='votes',
        aggfunc='sum'
    )
    
    # Flatten the columns
    pivoted_df.columns = [col for col in pivoted_df.columns]
    
    # Reset index
    pivoted_df.reset_index(inplace=True)
    
    # Append results
    results.append(pivoted_df)

# Concatenate all results
final_df = pd.concat(results, ignore_index=True)

final_df

### Count # of precincts for each county

In [ ]:
count_unique_df1 = final_df.groupby('county')['precinct'].nunique()
for county, count in count_unique_df1.items():
    print(f"{county}: {count}")

### Read AZ 2020 shapefile for potential re-use, in case where 2022 data is not available

In [ ]:
# Read the shapefile
shapefile_path = '../az_gen_20_prec/az_gen_20_prec.shp'
gdf = gpd.read_file(shapefile_path)

# Select only the desired columns
selected_columns = ['geometry', 'PRECINCTNA', 'COUNTY_NAM', 'UNIQUE_ID']
gdf_2020 = gdf[selected_columns]

def remove_special_characters(column):
    return column.str.replace(r'[^\w\s]', '', regex=True)

gdf_2020['PRECINCTNA'] = remove_special_characters(gdf_2020['PRECINCTNA'])
gdf_2020['COUNTY_NAM'] = remove_special_characters(gdf_2020['COUNTY_NAM'])
gdf_2020['UNIQUE_ID'] = remove_special_characters(gdf_2020['UNIQUE_ID'])

print(gdf_2020)


## Process precinct boundaries by county

In [ ]:
def plot_precincts_with_labels(gdf):
    """
    Plots a GeoDataFrame with color-coded precincts and labels, using parameters to customize the column for color, labels, figure size, colormap, edge color, line width, and label font size.
    """
    fig, ax = plt.subplots(figsize=(40, 40))
    gdf.plot(column='precinct', ax=ax, legend=True, cmap='tab20', edgecolor='black', linewidth=0.5)
    for x, y, label in zip(gdf.geometry.centroid.x, gdf.geometry.centroid.y, gdf['precinct']):
        ax.text(x, y, label, fontsize=15, ha='center')
    plt.show()

In [ ]:
def plot_precincts(gdf, col_name):
    """
    Plots a GeoDataFrame with color-coded precincts and labels, using parameters to customize the column for color, labels, figure size, colormap, edge color, line width, and label font size.
    """
    fig, ax = plt.subplots(figsize=(40, 40))
    gdf.plot(column=col_name, ax=ax, legend=True, cmap='tab20', edgecolor='black', linewidth=0.5)
    for x, y, label in zip(gdf.geometry.centroid.x, gdf.geometry.centroid.y, gdf[col_name]):
        ax.text(x, y, label, fontsize=15, ha='center')
    plt.show()

### Coconino County

In [ ]:
# Filter rows 
coconino_rows = final_df[final_df['county'].str.lower() == 'coconino']

# Display the filtered rows
print(coconino_rows.head(5))

In [ ]:
# Read in the GeoJSON file
geojson_file_Cococino = './precinct_boundaries/Cococino.geojson'
gdf_Cococino = gpd.read_file(geojson_file_Cococino)

print(len(gdf_Cococino))

final_df_Coconino = coconino_rows

# Extract the numerical part from the precinct values
final_df_Coconino['precinct'] = final_df_Coconino['precinct'].str.extract('(\d+)')[0].astype(int)

gdf_Cococino['VOTENUM'] = gdf_Cococino['VOTENUM'].astype(int)


# Merge the DataFrames on precinct_num and VOTENUM
merged_df = final_df_Coconino.merge(gdf_Cococino[['VOTENUM', 'geometry']], left_on='precinct', right_on='VOTENUM', how='right')

# Convert final_df to a GeoDataFrame
gdf_Cococino_final = gpd.GeoDataFrame(merged_df, geometry='geometry')
final_df_Coconino['precinct'] = final_df_Coconino['precinct'].astype(str)

plot_precincts_with_labels(gdf_Cococino_final)

In [ ]:
gdf_Cococino_final.drop(columns=['VOTENUM'], inplace=True)
gdf_Cococino_final.head(5)

### Apache County


In [ ]:
apache_2020 = gdf_2020[gdf_2020['COUNTY_NAM'] == 'Apache']

In [ ]:
apache_rows = final_df[final_df['county'].str.lower() == 'apache']
apache_rows['precinct'] = apache_rows['precinct'].str.upper()
print(apache_rows)

In [ ]:
# Merge the dataframes on UNIQUE_ID and precinct
gdf_Apache_final = apache_rows.merge(apache_2020[['UNIQUE_ID', 'geometry']], 
                                       left_on='precinct', 
                                       right_on='UNIQUE_ID', 
                                       how='right')

gdf_Apache_final = gdf_Apache_final.drop(columns=['UNIQUE_ID'])

gdf_Apache_final = gpd.GeoDataFrame(gdf_Apache_final, geometry='geometry')


plot_precincts_with_labels(gdf_Apache_final)
gdf_Apache_final.head(5)

### Cochise County

In [ ]:
cochise_rows = final_df[final_df['county'].str.lower() == 'cochise']
cochise_rows['precinct'] = remove_special_characters(cochise_rows['precinct'])
cochise_rows['precinct'] = cochise_rows['precinct'].str.extract('(\d+)')[0].astype(str).str.lstrip('0').astype(int)
print(cochise_rows.head(5))

In [ ]:
# Read the shapefile
shapefile_path_Cochise = './precinct_boundaries/Chochise/Election_Precinct_2022.shp'
gdf_Cochise = gpd.read_file(shapefile_path_Cochise)

In [ ]:
# Merge the dataframes
gdf_Cochise_final = cochise_rows.merge(gdf_Cochise[['prct_num', 'geometry']], 
                                       left_on='precinct', 
                                       right_on='prct_num', 
                                       how='right')

# Drop the duplicate col
gdf_Cochise_final = gdf_Cochise_final.drop(columns=['prct_num'])
gdf_Cochise_final = gpd.GeoDataFrame(gdf_Cochise_final, geometry='geometry')

plot_precincts_with_labels(gdf_Cochise_final)


### Gila County

In [ ]:
gila_rows = final_df[final_df['county'].str.lower() == 'gila']
gila_rows['precinct'] = remove_special_characters(gila_rows['precinct'])
def clean_precinct(precinct):
    precinct = precinct.replace(' ', '')
    precinct = precinct[:-3]
    return precinct

gila_rows['precinct'] = gila_rows['precinct'].apply(lambda x: clean_precinct(x))
sorted(gila_rows['precinct'])

mapping = {
   'MiamiNo1': 'Miami1',
   'MiamiNo3': 'Miami3'
}

# Replace old candidate names with new names
gila_rows['precinct'] = gila_rows['precinct'].replace(mapping)


In [ ]:
# Read the shapefile
shapefile_path_Gila = './precinct_boundaries/Gela/GilaPrecincts_Dec2022.shp'
gdf_Gila = gpd.read_file(shapefile_path_Gila)
gdf_Gila['NAME20'] = remove_special_characters(gdf_Gila['NAME20'])
gdf_Gila['NAME20'] = gdf_Gila['NAME20'].str.replace(' ', '')
gdf_Gila['NAME20']
plot_precincts(gdf_Gila,'NAME20')

In [ ]:
# Merge the dataframes
gdf_Gila_final = gila_rows.merge(gdf_Gila[['NAME20', 'geometry']], 
                                       left_on='precinct', 
                                       right_on='NAME20', 
                                       how='right')

# Drop the duplicate col
gdf_Gila_final = gdf_Gila_final.drop(columns=['NAME20'])
gdf_Gila_final = gpd.GeoDataFrame(gdf_Gila_final, geometry='geometry')

plot_precincts_with_labels(gdf_Gila_final)

## Graham County

In [ ]:
geojson_file_Graham = './precinct_boundaries/Graham.geojson'
gdf_Graham = gpd.read_file(geojson_file_Graham)
if len(gdf_Graham) != 22:
    raise ValueError("The GeoDataFrame must have exactly 22 rows.")

# Assign values from 1 to 22 to the 'name' column
gdf_Graham['NAME'] = range(1, 23)
print(gdf_Graham)


In [ ]:
graham_rows = final_df[final_df['county'].str.lower() == 'graham']
graham_rows['precinct'] = graham_rows['precinct'].str.extract('(\d+)')[0].astype(int)
print(graham_rows)

In [ ]:
# Merge the dataframes
gdf_Graham_final = graham_rows.merge(gdf_Graham[['NAME', 'geometry']], 
                                       left_on='precinct', 
                                       right_on='NAME', 
                                       how='right')

# Drop the duplicate col
gdf_Graham_final = gdf_Graham_final.drop(columns=['NAME'])

gdf_Graham_final = gpd.GeoDataFrame(gdf_Graham_final, geometry='geometry')

plot_precincts_with_labels(gdf_Graham_final)

## Greenlee County
Using 2020 boundaries(2022 boundaries provided are school districts not precinct)

In [ ]:
greenlee_rows = final_df[final_df['county'].str.lower() == 'greenlee']
greenlee_rows['precinct'] = greenlee_rows['precinct'].str[4:].str.upper()
greenlee_rows['precinct'] = remove_special_characters(greenlee_rows['precinct'])

print(greenlee_rows)

In [ ]:
greenlee_2020 = gdf_2020[gdf_2020['COUNTY_NAM'] == 'Greenlee']
print(greenlee_2020)

In [ ]:
# Merge the dataframes
gdf_Greenlee_final = greenlee_rows.merge(greenlee_2020[['PRECINCTNA', 'geometry']], 
                                       left_on='precinct', 
                                       right_on='PRECINCTNA', 
                                       how='right')

# Drop the duplicate col
gdf_Greenlee_final = gdf_Greenlee_final.drop(columns=['PRECINCTNA'])

gdf_Greenlee_final = gpd.GeoDataFrame(gdf_Greenlee_final, geometry='geometry')

plot_precincts_with_labels(gdf_Greenlee_final)

### La Paz County
Using 2020 boundaries

In [ ]:
la_paz_rows = final_df[final_df['county'].str.lower() == 'la paz']
la_paz_rows['precinct'] = la_paz_rows['precinct'].str.upper()

print(la_paz_rows)

In [ ]:
la_paz_2020 = gdf_2020[gdf_2020['COUNTY_NAM'] == 'La Paz']
print(la_paz_2020)

In [ ]:
# Merge the dataframes
gdf_La_Paz_final = la_paz_rows.merge(la_paz_2020[['UNIQUE_ID', 'geometry']], 
                                       left_on='precinct', 
                                       right_on='UNIQUE_ID', 
                                       how='left')

# Drop the duplicate col
gdf_La_Paz_final = gdf_La_Paz_final.drop(columns=['UNIQUE_ID'])

gdf_La_Paz_final = gpd.GeoDataFrame(gdf_La_Paz_final, geometry='geometry')

plot_precincts_with_labels(gdf_La_Paz_final)

### Maricopa County

In [ ]:
geojson_file_Maricopa = './precinct_boundaries/Maricopa.geojson'
gdf_Maricopa = gpd.read_file(geojson_file_Maricopa)

print(sorted(gdf_Maricopa['PctNum']))
print(len(gdf_Maricopa['PctNum']))
plot_precincts(gdf_Maricopa, 'PctNum')

In [ ]:
maricopa_rows = final_df[final_df['county'].str.lower() == 'maricopa']
maricopa_rows['precinct'] = maricopa_rows['precinct'].str.lower().str[1:4].str.lstrip('0')
print(len(maricopa_rows['precinct']))

The election results shows that Maricopa has 904 precincts, but precincts boyndaries shows that it has 935 precincts.
The official website mentioned that there are some precincts with no voters


In [ ]:
gdf_Maricopa['PctNum'] = gdf_Maricopa['PctNum'].astype(str)
maricopa_rows['precinct'] = maricopa_rows['precinct'].astype(str)

gdf_Maricopa_final = pd.merge(maricopa_rows, gdf_Maricopa[['PctNum', 'geometry']], how='right', left_on='precinct', right_on='PctNum')

# Reindex to ensure all columns from maricopa_rows are included, with geometry column added
gdf_Maricopa_final = gdf_Maricopa_final.reindex(columns=list(maricopa_rows.columns) + ['geometry'])


gdf_Maricopa_final = gpd.GeoDataFrame(gdf_Maricopa_final, geometry='geometry')

plot_precincts_with_labels(gdf_Maricopa_final)

### Mohave County

In [ ]:
# Read the shapefile
shapefile_path_Mohave = './precinct_boundaries/Mohave/Voting_Precincts_2023.shp'
gdf_Mohave = gpd.read_file(shapefile_path_Mohave)

print(gdf_Mohave)


In [ ]:
mohave_rows = final_df[final_df['county'].str.lower() == 'mohave']
mohave_rows['precinct'] = mohave_rows['precinct'].str[4:].str.upper()
mohave_rows['precinct'] = remove_special_characters(mohave_rows['precinct'])
print(mohave_rows)

In [ ]:
gdf_Mohave_final = mohave_rows.merge(gdf_Mohave[['NAME', 'geometry']], 
                                       left_on='precinct', 
                                       right_on='NAME', 
                                       how='right')

gdf_Mohave_final = gdf_Mohave_final.drop(columns=['NAME'])

gdf_Mohave_final = gpd.GeoDataFrame(gdf_Mohave_final, geometry='geometry')

plot_precincts_with_labels(gdf_Mohave_final)

### Navajo County

In [ ]:
navajo_rows = final_df[final_df['county'].str.lower() == 'navajo']
print(navajo_rows)

In [ ]:
navajo_2020 = gdf_2020[gdf_2020['COUNTY_NAM'] == 'Navajo']
print(navajo_2020)

In [ ]:
# Convert both columns to uppercase
navajo_rows['precinct'] = navajo_rows['precinct'].str.upper()
navajo_2020['UNIQUE_ID'] = navajo_2020['UNIQUE_ID'].str.upper()

# Check for matching values again
print(navajo_rows['precinct'].unique())
print(navajo_2020['UNIQUE_ID'].unique())

# Merge the dataframes on UNIQUE_ID and precinct
gdf_Navajo_final = navajo_rows.merge(navajo_2020[['UNIQUE_ID', 'geometry']], 
                                     left_on='precinct', 
                                     right_on='UNIQUE_ID', 
                                     how='right')

# Drop the duplicate UNIQUE_ID column from the merged DataFrame
gdf_Navajo_final = gdf_Navajo_final.drop(columns=['UNIQUE_ID'])
gdf_Navajo_final = gpd.GeoDataFrame(gdf_Navajo_final, geometry='geometry')

plot_precincts_with_labels(gdf_Navajo_final)


### Pima County

In [ ]:
# Read the shapefile
shapefile_path_Pima = './precinct_boundaries/Pima/Districts_-_Voter_Precincts.shp'
gdf_Pima = gpd.read_file(shapefile_path_Pima)
print(gdf_Pima)
plot_precincts(gdf_Pima, 'OBJECTID')

In [ ]:
pima_rows = final_df[final_df['county'].str.lower() == 'pima']
print(pima_rows)
print(len(pima_rows['precinct'].unique()))

Official website mentioned that "2/28/22 - Elections proposed many boundary adjustments, including the creation of new precincts, in order to align with the new Congressional and Legislative District boundaries created by the Arizona Independent Redistricting Commission. Number of precincts increased from 249 to 278. Board of Supervisors approved changes on 2/15/2022". However, the election result only has 266 precints from both sources.

In [ ]:
# Convert the 'precinct' column in pinal_rows to string
pima_rows['precinct'] = pima_rows['precinct'].astype(str)

# Convert the 'PRECINCT' column in gdf_Pima to string
gdf_Pima['PRECINCT'] = gdf_Pima['PRECINCT'].astype(str)

# Perform the left join merge
gdf_Pima_final = pima_rows.merge(gdf_Pima[['PRECINCT', 'geometry']], 
                                  left_on='precinct', 
                                  right_on='PRECINCT', 
                                  how='right')

# Drop the duplicate 'PRECINCT' column from the merged DataFrame
gdf_Pima_final = gdf_Pima_final.drop(columns=['PRECINCT'])

gdf_Pima_final = gpd.GeoDataFrame(gdf_Pima_final, geometry='geometry')

plot_precincts_with_labels(gdf_Pima_final)


### Pinal County

In [ ]:
# Read the shapefile
shapefile_path_Pinal = './precinct_boundaries/Pinal/Voter_Precincts.shp'
gdf_Pinal = gpd.read_file(shapefile_path_Pinal)
print(gdf_Pinal)
plot_precincts(gdf_Pinal, 'NAME')

In [ ]:
# Get unique values in the 'precinct' column
unique_name = gdf_Pinal['NAME'].unique()

# Sort the unique values in alphabetical order
unique_name_sorted = sorted(unique_name)
print(len(unique_name_sorted))

# Print each unique value line by line
for precinct in unique_name_sorted:
    print(precinct)

In [ ]:
pinal_rows = final_df[final_df['county'].str.lower() == 'pinal']
pinal_rows['precinct'] = pinal_rows['precinct'].str[3:].str.upper()
print(pinal_rows)

There are 5 precincts that do not match, 4 of them are minor name differences and 1 is no vote precint. eg. Apache Junction Shouth -> Apache JCT South. And SAN CARLOS COMMUNITY in the shapefile does not match with any election precincts, which I still include by filled it with null values.

In [ ]:
value_mapping = {
    'APACHE JCT SOUTH': 'APACHE JUNCTION SOUTH',
    'APACHE JCT SUPERSTITION': 'APACHE JUNCTION SUPERSTITION',
    'COYOTE': 'COYOTE RANCH',
    'SADDLEBROOKE RANCHE': 'SADDLEBROOKE RANCH'
}

pinal_rows['precinct'] = pinal_rows['precinct'].replace(value_mapping)

In [ ]:
pinal_rows['precinct'] = pinal_rows['precinct'].str.strip()
unique_precincts = pinal_rows['precinct'].unique()

# Sort the unique values in alphabetical order
unique_precincts_sorted = sorted(unique_precincts)
print(len(unique_precincts_sorted))

# Print each unique value line by line
for precinct in unique_precincts_sorted:
    print(precinct)

In [ ]:
# Perform the right merge to keep all geometries from gdf_Pinal
gdf_Pinal_final = pinal_rows.merge(
    gdf_Pinal[['NAME', 'geometry']], 
    left_on='precinct', 
    right_on='NAME', 
    how='right'
)

# # Drop the 'NAME' column to avoid duplication and confusion
# gdf_Pinal_final = gdf_Pinal_final.drop(columns=['NAME'])

# Convert the merged DataFrame to a GeoDataFrame
gdf_Pinal_final = gpd.GeoDataFrame(gdf_Pinal_final, geometry='geometry')


# Plot the precincts with labels
plot_precincts(gdf_Pinal_final, 'NAME')


### Santa Cruz County

In [ ]:
santa_cruz_2020 = gdf_2020[gdf_2020['COUNTY_NAM'] == 'Santa Cruz']
print(santa_cruz_2020)
print(len(santa_cruz_2020))
print(sorted(santa_cruz_2020['UNIQUE_ID']))
plot_precincts(santa_cruz_2020, 'UNIQUE_ID')

In [ ]:
santa_cruz_rows = final_df[final_df['county'].str.lower() == 'santa cruz']
# print(santa_cruz_rows)

# After checking multiple sources, the election result marked Calabas as 16 and 17 but instead there is only 1 Calabas precinct. So drop it:
santa_cruz_rows = santa_cruz_rows[santa_cruz_rows['precinct'] != 'Calabasas 16']

# Also, Beca 19.6 should be Beca 19 19.6 for consistency
santa_cruz_rows.loc[santa_cruz_rows['precinct'] == 'Baca 19.6', 'precinct'] = 'Baca 19 19.6'

# Function to determine the simplified name for grouping
def get_simplified_name(precinct):
    parts = precinct.split()
    if len(parts) > 2:
        if any(char.isdigit() for char in parts[2]):
            return ' '.join(parts[:2])
    return precinct
    
# Create a new column with the simplified name for grouping
santa_cruz_rows['simplified_precinct'] = santa_cruz_rows['precinct'].apply(get_simplified_name)

# Define aggregation functions for all columns
aggregation_functions = {col: 'first' for col in santa_cruz_rows.columns if col != 'simplified_precinct'}

# Group by the simplified name and aggregate
grouped_df = santa_cruz_rows.groupby('simplified_precinct').agg(aggregation_functions).reset_index(drop=True)


def simplify_precinct_grouped(precinct):
    parts = precinct.split()
    if len(parts) > 1:
        return ' '.join(parts[:2])
    return precinct

# Apply the function to the 'precinct' column
grouped_df['precinct'] = grouped_df['precinct'].apply(simplify_precinct_grouped)

mapping = {
    'Peck Canyon': 'Peck Canyon 21', 
    'Rio Rico': 'Rio Rico 7',
    'Santa Cruz': 'Santa Cruz 13',
    'Lake Patagonia': 'Lake Patagonia 24',
}
grouped_df['precinct'] = grouped_df['precinct'].replace(mapping)


# Display the final DataFrame
print(len(grouped_df))
sorted(grouped_df['precinct'])

In [ ]:
# Merge the dataframes on UNIQUE_ID and precinct
gdf_Santa_Cruz_final = grouped_df.merge(santa_cruz_2020[['PRECINCTNA', 'geometry']], 
                                       left_on='precinct', 
                                       right_on='PRECINCTNA', 
                                       how='right')

# Drop the duplicate UNIQUE_ID column from the merged DataFrame
gdf_Santa_Cruz_final = gdf_Santa_Cruz_final.drop(columns=['PRECINCTNA'])


gdf_Santa_Cruz_final = gpd.GeoDataFrame(gdf_Santa_Cruz_final, geometry='geometry')

plot_precincts_with_labels(gdf_Santa_Cruz_final)


### Yavapai County

In [ ]:
# Read the shapefile
shapefile_path_Yavapai = './precinct_boundaries/Yavapai/YC_ElectionPrecincts.shp'
gdf_Yavapai = gpd.read_file(shapefile_path_Yavapai)

print(sorted(gdf_Yavapai['PRECINCT']))
plot_precincts(gdf_Yavapai, 'PRECINCT')



In [ ]:
yavapai_rows = final_df[final_df['county'].str.lower() == 'yavapai']
yavapai_rows['precinct'] = yavapai_rows['precinct'].str.replace(r'[^a-zA-Z\s]', '', regex=True)
print(sorted(yavapai_rows['precinct']))
print(yavapai_rows)

In [ ]:
# Clean the 'precinct' values to keep only letters and convert to uppercase
yavapai_rows['precinct'] = yavapai_rows['precinct'].str.replace(r'[^a-zA-Z\s]', '', regex=True).str.upper().str.strip()
gdf_Yavapai['PRECINCT'] = gdf_Yavapai['PRECINCT'].str.upper().str.strip()

gdf_Yavapai_final = yavapai_rows.merge(gdf_Yavapai[['PRECINCT', 'geometry']], 
                                       left_on='precinct', 
                                       right_on='PRECINCT', 
                                       how='right')

# gdf_Yavapai_final = gdf_Yavapai_final.drop(columns=['PRECINCT'])

gdf_Yavapai_final = gpd.GeoDataFrame(gdf_Yavapai_final, geometry='geometry')


# Plot the precincts with labels
plot_precincts(gdf_Yavapai_final, 'PRECINCT')


### Yuma County
Using 2020 boundaries

In [ ]:
yuma_rows = final_df[final_df['county'].str.lower() == 'yuma']
yuma_rows['precinct'] = yuma_rows['precinct'].str.split('.').str[0]
yuma_rows.loc[yuma_rows['precinct'] == '25', 'precinct'] = '025'
print(yuma_rows)
print(len(yuma_rows))


In [ ]:
# Read the shapefile
shapefile_path_Yuma = './precinct_boundaries/yuma_2022/fwd2022voterprecinctmapasshapefiles/Voting_Pricincts_03_2023.shp'
gdf_Yuma_2022 = gpd.read_file(shapefile_path_Yuma)

print(gdf_Yuma_2022)
plot_precincts(gdf_Yuma_2022, 'Precinct')

After checking with voter file on L2, it is confirmed that districts 21 and 44 have no voters.

In [ ]:
gdf_Yuma_final = yuma_rows.merge(gdf_Yuma_2022[['CODE', 'geometry']], 
                                       left_on='precinct', 
                                       right_on='CODE', 
                                       how='right')

# Drop the duplicate UNIQUE_ID column from the merged DataFrame
gdf_Yuma_final = gdf_Yuma_final.drop(columns=['CODE'])

# Display the final DataFrame
print(gdf_Yuma_final)

## Combine county level PBER into state level

In [ ]:
import geopandas as gpd
import pandas as pd

# List of GeoDataFrames
gdf_list = [
    gdf_Apache_final, gdf_Cochise_final, gdf_Cococino_final, gdf_Gila_final,
    gdf_Graham_final, gdf_Greenlee_final, gdf_Maricopa_final, gdf_La_Paz_final,
    gdf_Mohave_final, gdf_Navajo_final, gdf_Pima_final, gdf_Pinal_final,
    gdf_Santa_Cruz_final, gdf_Yavapai_final, gdf_Yuma_final
]

target_crs = "EPSG:4326"
valid_gdfs = []

for gdf in gdf_list:
    if not isinstance(gdf, gpd.GeoDataFrame):
        try:
            gdf = gpd.GeoDataFrame(gdf, geometry='geometry')
        except KeyError:
            print(f"Missing geometry column in DataFrame: {gdf}")
            continue

    if gdf.crs is None:
        gdf.set_crs(target_crs, inplace=True)

    valid_gdfs.append(gdf.to_crs(target_crs))

# Combine reprojected GeoDataFrames into a single GeoDataFrame
combined_gdf = pd.concat(valid_gdfs, ignore_index=True)

combined_gdf


In [ ]:
# Remove duplicated columns
def remove_duplicated_columns(gdf):
    duplicates = gdf.columns[gdf.columns.duplicated()].unique()
    gdf = gdf.loc[:, ~gdf.columns.duplicated()]
    return gdf

# Clean field names to ensure they are unique and valid for shapefiles
def clean_field_names(gdf):
    new_columns = {}
    seen_columns = set()
    for col in gdf.columns:
        new_col = ''.join([c if ord(c) < 128 else '' for c in col])[:10]
        # Ensure the column name is unique
        if new_col in seen_columns:
            counter = 1
            new_col_temp = new_col
            while new_col_temp in seen_columns:
                new_col_temp = f"{new_col[:8]}{counter}"
                counter += 1
            new_col = new_col_temp
        seen_columns.add(new_col)
        new_columns[col] = new_col
    gdf.rename(columns=new_columns, inplace=True)
    return gdf

# Remove duplicated columns
combined_gdf = remove_duplicated_columns(combined_gdf)

# Clean the field names in your GeoDataFrame
cleaned_gdf = clean_field_names(combined_gdf)

# Check for duplicated columns after cleaning
if cleaned_gdf.columns.duplicated().any():
    print("There are still duplicated column names after cleaning.")
else:
    print("No duplicated columns after cleaning.")

# Save the GeoDataFrame to a shapefile
cleaned_gdf.to_file("combined_gdf.shp")


In [ ]:
# Read the shapefile
shapefile_path_final = './combined_gdf.shp'
shapefile_final = gpd.read_file(shapefile_path_final)

ax = shapefile_final.plot()

# Create a larger plot
fig, ax = plt.subplots(figsize=(20, 20))

# Plot the geometries
shapefile_final.plot(ax=ax)

# Add labels to each geometry
for x, y, label in zip(shapefile_final.geometry.centroid.x, shapefile_final.geometry.centroid.y, shapefile_final['county']):
    ax.text(x, y, label, fontsize=5, ha='center')  # Adjust font size as needed

plt.show()